In [6]:
from collections import OrderedDict

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch import optim


class CNN6(nn.Module):
    def __init__(self):
        super(CNN6, self).__init__()
        act = nn.LeakyReLU(negative_slope=0.2)
        self.body = nn.ModuleList([
            nn.Sequential(OrderedDict([
                ('layer', nn.Conv2d(3, 12, kernel_size=4, padding=2, stride=2, bias=False)),
                ('act', act)
            ])),
            nn.Sequential(OrderedDict([
                ('layer', nn.Conv2d(12, 36, kernel_size=3, padding=1, stride=2, bias=False)),
                ('act', act)
            ])),
            nn.Sequential(OrderedDict([
                ('layer', nn.Conv2d(36, 36, kernel_size=3, padding=1, stride=1, bias=False)),
                ('act', act)
            ])),
            nn.Sequential(OrderedDict([
                ('layer', nn.Conv2d(36, 36, kernel_size=3, padding=1, stride=1, bias=False)),
                ('act', act)
            ])),
            nn.Sequential(OrderedDict([
                ('layer', nn.Conv2d(36, 64, kernel_size=3, padding=1, stride=2, bias=False)),
                ('act', act)
            ])),
            nn.Sequential(OrderedDict([
                ('layer', nn.Conv2d(64, 128, kernel_size=3, padding=1, stride=1, bias=False)),
                ('act', act)
            ])),
            nn.Sequential(OrderedDict([
                ('layer', nn.Linear(3200, 10, bias=False)),
                ('act', nn.Identity())
            ]))
        ])

    def forward(self, x):
        for layer in self.body:
            if isinstance(layer.layer, nn.Linear):
                x = x.flatten(1)
            x = layer(x)
        return x


if __name__ == '__main__':
    #  **由于torchvision的datasets的输出是[0,1]的PILImage，所以我们先先归一化为[-1,1]的Tensor**
    #  首先定义了一个变换transform，利用的是上面提到的transforms模块中的Compose( )
    #  把多个变换组合在一起，可以看到这里面组合了ToTensor和Normalize这两个变换
    torch.manual_seed(seed=10000)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    transform = transforms.Compose(
        [transforms.ToTensor(),
         transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

    # 定义了我们的训练集，名字就叫trainset，至于后面这一堆，其实就是一个类：
    # torchvision.datasets.CIFAR10( )也是封装好了的，就在我前面提到的torchvision.datasets
    # 模块中,不必深究，如果想深究就看我这段代码后面贴的图1，其实就是在下载数据
    # （不翻墙可能会慢一点吧）然后进行变换，可以看到transform就是我们上面定义的transform
    trainset = torchvision.datasets.CIFAR10(root='cifar-10-batches-py/', train=True,
                                            download=True, transform=transform)
    # trainloader其实是一个比较重要的东西，我们后面就是通过trainloader把数据传入网
    # 络，当然这里的trainloader其实是个变量名，可以随便取，重点是他是由后面的
    # torch.utils.data.DataLoader()定义的，这个东西来源于torch.utils.data模块，
    #  网页链接http://pytorch.org/docs/0.3.0/data.html，这个类可见我后面图2
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=128,
                                              shuffle=True, num_workers=2)
    # 对于测试集的操作和训练集一样，我就不赘述了
    testset = torchvision.datasets.CIFAR10(root='cifar-10-batches-py/', train=False,
                                           download=True, transform=transform)
    testloader = torch.utils.data.DataLoader(testset, batch_size=128,
                                             shuffle=False, num_workers=2)
    # 类别信息也是需要我们给定的
    classes = ('plane', 'car', 'bird', 'cat',
               'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

    model = CNN6()
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    epochs = 10
    for epoch in range(epochs):
        running_loss = 0
        num_correct_prediction = 0
        for images, labels in trainloader:
            images = images.to(device)
            labels = labels.to(device)
            model = model.to(device)
            output = model(images)
            _, predicted = torch.max(F.log_softmax(output.data), 1)
            loss = criterion(output, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            num_correct_prediction += (predicted == labels).sum().item()
            running_loss += loss.item()
        else:
            print(f"Epoch:{epoch}, Training loss:{running_loss}, Training accuracy:{num_correct_prediction / 50000}")

    model.eval()
    test_loss = 0
    num_correct_prediction = 0
    with torch.no_grad():
        for images, labels in testloader:
            images = images.to(device)
            labels = labels.to(device)
            model = model.to(device)
            output = model(images)
            _, predicted = torch.max(F.log_softmax(output.data), 1)
            loss = criterion(output, labels)
            test_loss += loss.item()
            num_correct_prediction += (predicted == labels).sum().item()
    print(f"Test loss:{test_loss}, Test accuracy:{num_correct_prediction / 10000}")


Files already downloaded and verified
Files already downloaded and verified


C:\Users\SF230210\AppData\Local\Temp\ipykernel_26808\3032957443.py:97: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  _, predicted = torch.max(F.log_softmax(output.data), 1)


Epoch:0, Training loss:665.2954579591751, Training accuracy:0.38742
Epoch:1, Training loss:524.9937934875488, Training accuracy:0.5222
Epoch:2, Training loss:464.9594092965126, Training accuracy:0.58054
Epoch:3, Training loss:419.4039323925972, Training accuracy:0.62242
Epoch:4, Training loss:382.1145574450493, Training accuracy:0.65868
Epoch:5, Training loss:348.7179926633835, Training accuracy:0.68938
Epoch:6, Training loss:321.81937754154205, Training accuracy:0.71548
Epoch:7, Training loss:296.2168590128422, Training accuracy:0.73842
Epoch:8, Training loss:271.64105728268623, Training accuracy:0.75822
Epoch:9, Training loss:250.2487301826477, Training accuracy:0.77598


C:\Users\SF230210\AppData\Local\Temp\ipykernel_26808\3032957443.py:116: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  _, predicted = torch.max(F.log_softmax(output.data), 1)


Test loss:74.82130819559097, Test accuracy:0.6893


In [ ]:
###动态调整学习率scheduler = StepLR(optimizer, step_size=30, gamma=0.1)
###scheduler = MultiStepLR(optimizer, milestones=[30,80], gamma=0.1)
###有必要，使模型不那么敏感